In [1]:
import numpy as np
import pandas as pd
import os
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [3]:
# pd.show_versions()

In [4]:
import gc
from math import sqrt
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

import math as mt
from math import *
import matplotlib as mlp
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import seaborn as sns
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_predict


import joblib
import random
import itertools
import scipy as sp

from itertools import combinations
import warnings; warnings.filterwarnings('ignore')

In [5]:
# df_train = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/train.parquet')

In [6]:
# cat_feats = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

# # D_* = Delinquency variables
# # S_* = Spend variables
# # P_* = Payment variables
# # B_* = Balance variables
# # R_* = Risk variables

# D_feats = [col for col in df_train[df_train.customer_ID=='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'].columns if 'D_' in col]
# S_feats = [col for col in df_train[df_train.customer_ID=='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'].columns if 'S_' in col]
# P_feats = [col for col in df_train[df_train.customer_ID=='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'].columns if 'P_' in col]
# B_feats = [col for col in df_train[df_train.customer_ID=='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'].columns if 'B_' in col]
# R_feats = [col for col in df_train[df_train.customer_ID=='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'].columns if 'R_' in col]

In [7]:
# len(D_feats), len(S_feats), len(P_feats), len(B_feats), len(R_feats)

In [8]:
# df_train[df_train.customer_ID=='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'][P_feats]

In [9]:
cat_cols = ['B_30','B_38','D_114','D_116','D_117','D_120','D_126','D_63','D_64','D_66','D_68']

more_cats = ['S_8','S_11','S_13','S_15','B_4','B_16','B_19','B_20','B_22','B_33','B_41','R_3','R_5','R_8',
             'R_9','R_10','R_11','R_13','R_16','R_17','R_18','R_20','R_26','R_9','D_39','D_44','D_49','D_51',
             'D_59','D_70','D_72','D_74','D_75','D_78','D_79','D_80','D_81','D_82','D_83','D_84','D_89',
             'D_91','D_92','D_103','D_106','D_107','D_108','D_109','D_111','D_113','D_122','D_123','D_123',
             'D_124','D_125','D_129','D_135','D_136','D_137','D_138','D_139','D_140','D_143','D_145']

binary_cats = ['S_6','S_18','S_20','B_31','B_32','R_2','R_4','R_15','R_19','R_21','R_22','R_23','R_24','R_25','R_28',
              'D_86','D_87','D_93','D_94','D_96','D_127']

In [10]:
len(cat_cols), len(more_cats), len(binary_cats)

(11, 64, 21)

In [11]:
# cat_feats = cat_cols + more_cats + binary_cats
cat_feats = cat_cols + binary_cats
len(cat_feats)

32

In [12]:
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    df1 = np.concatenate(df1, axis = 0)
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    df1['customer_ID'] = customer_ids
    return df1

In [13]:
def read_preprocess_data(cat_feats):
    train = pd.read_parquet('../src/data/raw/train.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    
    cat_features = cat_feats.copy()
    
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    
    num_features = [col for col in features if col not in cat_features]
    
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)

    # Lag Features
    for col in train_num_agg:
        for col_2 in ['first', 'mean', 'std', 'min', 'max']:
            if 'last' in col and col.replace('last', col_2) in train_num_agg:
                train_num_agg[col + '_lag_sub'] = train_num_agg[col] - train_num_agg[col.replace('last', col_2)]
                train_num_agg[col + '_lag_div'] = train_num_agg[col] / train_num_agg[col.replace('last', col_2)]
    
    train_labels = pd.read_csv('../src/data/raw/train_labels.csv')
    
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID')
    del train_cat_agg, train_num_agg
    gc.collect()
    
    train = train.merge(train_diff, how = 'inner', on = 'customer_ID')
    del train_diff
    gc.collect()
    
    train = train.merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_labels
    gc.collect()
    
    train.to_parquet('../src/data/processed/train_fe_loaded.parquet')
    del train
    gc.collect()
    
    test = pd.read_parquet('../src/data/raw/test.parquet')
    
    print('Starting test feature engineer...')
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)

    # Lag Features
    for col in test_num_agg:
        for col_2 in ['first', 'mean', 'std', 'min', 'max']:
            if 'last' in col and col.replace('last', col_2) in test_num_agg:
                test_num_agg[col + '_lag_sub'] = test_num_agg[col] - test_num_agg[col.replace('last', col_2)]
                test_num_agg[col + '_lag_div'] = test_num_agg[col] / test_num_agg[col.replace('last', col_2)]
    
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    # Get the difference
    
    test_diff = get_difference(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg
    gc.collect()    
    test = test.merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_diff
    gc.collect()    
    test.to_parquet('../src/data/processed/test_fe_loaded.parquet')
    
# Read & Preprocess Data
read_preprocess_data(cat_feats)

Starting training feature engineer...


100%|███████████████████████████████████████████████████████| 458913/458913 [12:38<00:00, 605.13it/s]


Starting test feature engineer...


100%|███████████████████████████████████████████████████████| 924621/924621 [25:52<00:00, 595.49it/s]


In [14]:
!kaggle datasets init -p ../src/data/processed

Data package template written to: ../src/data/processed/dataset-metadata.json


In [15]:
!kaggle datasets create -p ../src/data/processed

Starting upload for file test_fe_loaded.parquet
100%|██████████████████████████████████████| 2.76G/2.76G [01:10<00:00, 41.7MB/s]
Upload successful: test_fe_loaded.parquet (3GB)
Starting upload for file train_fe_loaded.parquet
100%|██████████████████████████████████████| 1.53G/1.53G [00:42<00:00, 38.9MB/s]
Upload successful: train_fe_loaded.parquet (2GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/tarique7/amex-fe-addcats
